# Multi-class FSA

1. Implement the FSA variable selection method with linear models for multi-class
classification with the Vapnik los
$L(\bf{u}, y) = \Sigma_{k\not=y} l(u_y-u_k)$

where $l(u)$ is the logistic loss described in class. Use the parameters $\lambda = 0.0001, \mu =
100, N^{iter} = 500$.
Take special care to normalize each column of the $X$ matrix to have zero mean
and variance $1$ and to use for normalizing the test set the same mean and standard
deviation that you used for normalizing the training set.
Assuming that the coefficient vector is a $p \times c$ matrix $W$, where $p$ is the number of
features and c is the number of classes, use the norm $||\bf{w}_j||, j = 1, ..., p$ of each row as
the criterion to select the variables in FSA.

Import dependencies

In [650]:
import numpy as np
import torch
from sklearn.metrics import accuracy_score

#### Normalization function

In [651]:
def normalize(train, *args):
    mean = np.average(train, axis=0)
    standard_deviation = np.std(train, axis=0)
    columns = train, *args
    return tuple(np.divide(column-mean, standard_deviation, where=standard_deviation!=0)
                 for column in columns)

#### Load data

In [652]:
def get_data():
    path = "data/satimage/"

    train_x = np.loadtxt(path + "X.dat")
    train_y_temp = np.loadtxt(path + "Y.dat", dtype=np.int)
    test_x = np.loadtxt(path + "Xtest.dat")
    test_y_temp = np.loadtxt(path + "Ytest.dat", dtype=np.int)

    train_x, test_x = normalize(train_x, test_x)

    train_y = np.zeros((train_y_temp.size,np.max(train_y_temp)))
    train_y[np.arange(train_y_temp.size),train_y_temp-1] = 1

    test_y = np.zeros((test_y_temp.size,train_y.shape[1]))
    test_y[np.arange(test_y_temp.size),test_y_temp-1] = 1

    return train_x, train_y, test_x, test_y

get_data()


(array([[ 1.6504898 ,  1.37065618,  1.24035744, ...,  1.04773951,
          0.83669871,  0.23370924],
        [ 1.06432744,  0.79853645,  0.40053984, ...,  0.69624747,
          0.29644556, -0.18971944],
        [ 1.06432744,  0.79853645,  0.16059195, ...,  0.69624747,
          0.29644556, -0.18971944],
        ...,
        [-0.10799728, -0.38971222,  0.52051378, ...,  0.74018398,
          0.29644556,  0.12785207],
        [ 0.11181361,  0.1383983 ,  0.52051378, ...,  0.34475544,
          0.29644556,  0.12785207],
        [ 0.11181361,  0.31443514,  0.04061801, ...,  0.34475544,
          0.05633305, -0.08386227]]),
 array([[0., 1., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        ...,
        [0., 0., 1., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 0., 1., 0., 0.]]),
 array([[ 0.77124626,  0.79853645,  0.16059195, ...,  1.04773951,
          0.83669871,  0.23370924],
        [ 0.47816508,  0.79853645,  0.16059195, ...,  0.87199349,
          

#### Calculate the next iteration's weights

In [653]:
def update_weights(w, x, y):
    learning_rate = 1/x.shape[0]
    learning_rate = .05
    return w + learning_rate * L(w, x, y)

def L(w, x, y):
    """
    Gradient of logistic loss function with sparsity-inducing penalty
    """
    s = 0.0001
    w = torch.from_numpy(w)
    x = torch.from_numpy(x)
    y = torch.from_numpy(y)

    w.requires_grad = True
    y_predicted = torch.matmul(x,w)
    difference = y - y_predicted
    difference[y==1]=0
    #print("before", torch.nansum(difference))
    #difference+= .0001
    #print(torch.log(torch.exp(difference)+1) - torch.log(torch.exp(difference)))
    print(torch.min(1+torch.exp(-difference)))
    loss = torch.log(1 + torch.exp(-difference))
    #print(loss)
    #loss = torch.log(torch.exp(difference)+1) - torch.log(torch.exp(difference))
    #print("after", torch.nansum(loss))
    loss = torch.sum(loss)
    #print(y_predicted)
    loss.backward()
    #print(loss.detach().numpy())
    gradient = w.grad + 2 * s * 2
    return gradient.numpy()

    #return x.T.dot(y-(1/(1+np.exp(-np.dot(x,w))))) + 2 * s * w

def predict(X,w):
    """
    Predictor function for logistic regression
    """
    z = X @ w
    z /= np.sum(z,axis=1).reshape(-1,1)
    result = np.zeros_like(z)
    result[np.arange(z.shape[0]), np.argmax(z, axis=1)] = 1
    return result

#### Calculate the number of parameters to keep

In [654]:
def calc_schedule(num_params, k, num_iterations):
    mu = 30
    return [round(k+(num_params-k)*max(0,(num_iterations-2*i)/(2*i*mu+num_iterations)))
            for i in range(1,num_iterations+1)]

#### Eliminate Parameters

In [655]:
def eliminate_parameters(parameters_to_keep, w, x_train, x_test):
    """
    Eliminates the parameters which have the smallest weights so that parameters_to_keep parameter remain
    """
    # list of indexes that would sort the array
    sorted_indexes = np.argsort(np.linalg.norm(w, axis=1))
    # index thresholds used to eliminate insignificant parameters
    threshold = w.shape[0] - parameters_to_keep
    # delete indexes smaller than threshold
    w = np.delete(w, sorted_indexes[sorted_indexes < threshold], axis=0)
    x_train = np.delete(x_train, sorted_indexes[sorted_indexes < threshold], axis=1)
    x_test = np.delete(x_test, sorted_indexes[sorted_indexes < threshold], axis=1)
    #print('after',w)
    return w, x_train, x_test


#### Train logistic regression model using FSA

In [656]:
def train_fsa(x_train, y_train, x_test, y_test, k):
    iterations = 500
    x_train, x_test = normalize(x_train, x_test)
    # append column of ones for the intercept
    x_train = np.hstack((np.ones((x_train.shape[0], 1)), x_train))
    x_test = np.hstack((np.ones((x_test.shape[0], 1)), x_test))
    # make sure data is binary 0,1 rather than -1,1

    w = np.zeros((x_train.shape[1], y_train.shape[1])) # initializing weights to 0
    schedule = calc_schedule(w.shape[0], k, iterations)
    train_scores = []
    test_scores = []
    train_scores.append(1 - accuracy_score(y_train, predict(x_train, w)))
    test_scores.append(1 - accuracy_score(y_test, predict(x_test, w)))
    for parameters_to_keep in schedule:
        if parameters_to_keep == 0:
            break
        w = update_weights(w, x_train, y_train)
        w, x_train, x_test = eliminate_parameters(parameters_to_keep, w, x_train, x_test)
        train_scores.append(1 - accuracy_score(y_train, predict(x_train, w)))
        test_scores.append(1 - accuracy_score(y_test, predict(x_test, w)))
    number_of_features = w.shape[0]
    return test_scores, train_scores, number_of_features

train_x, train_y, test_x, test_y = get_data()
k =  5
train_fsa(train_x, train_y, test_x, test_y, k)
print()

C:\Users\Oscar\anaconda3\envs\tensor_environemnt\lib\site-packages\ipykernel_launcher.py:41: RuntimeWarning: invalid value encountered in true_divide


tensor(2., dtype=torch.float64, grad_fn=<MinBackward1>)
tensor(1., dtype=torch.float64, grad_fn=<MinBackward1>)
tensor(nan, dtype=torch.float64, grad_fn=<MinBackward1>)
tensor(nan, dtype=torch.float64, grad_fn=<MinBackward1>)
tensor(nan, dtype=torch.float64, grad_fn=<MinBackward1>)
tensor(nan, dtype=torch.float64, grad_fn=<MinBackward1>)
tensor(nan, dtype=torch.float64, grad_fn=<MinBackward1>)
tensor(nan, dtype=torch.float64, grad_fn=<MinBackward1>)
tensor(nan, dtype=torch.float64, grad_fn=<MinBackward1>)
tensor(nan, dtype=torch.float64, grad_fn=<MinBackward1>)
tensor(nan, dtype=torch.float64, grad_fn=<MinBackward1>)
tensor(nan, dtype=torch.float64, grad_fn=<MinBackward1>)
tensor(nan, dtype=torch.float64, grad_fn=<MinBackward1>)
tensor(nan, dtype=torch.float64, grad_fn=<MinBackward1>)
tensor(nan, dtype=torch.float64, grad_fn=<MinBackward1>)
tensor(nan, dtype=torch.float64, grad_fn=<MinBackward1>)
tensor(nan, dtype=torch.float64, grad_fn=<MinBackward1>)
tensor(nan, dtype=torch.float64, 

a) Using the satimage data, train a multi-class FSA classifier on the training set,
starting with $\beta(0) = 0$ to select $k \in \{5, 9, 18, 27, 36\}$ features. For each $k$ find
an appropriate learning rate $\eta$ to obtain a small final loss value on the training
set. Plot the training loss vs iteration number for $k = 27$. (5 points)

b) Report in a table the misclassification errors on the training and test set for the
models obtained for all these $k$. Plot the misclassification error on the training
and test set vs $k$. (1 point)

c) Repeat points a) and b) using the cross-entropy loss function:
$L(\bf{u}, y) = −u_y + \ln(\Sigma_{k=1}^C \exp(u_k))$
adding the misclassification errors to the table from b). (4 points).
